**INITIALIZATION:**

In [1]:
#@ INITIALIZING NECESSARY PACKAGES AND DEPENDENCIES: 
import numpy as np

**PERCEPTRON:**

In [2]:
#@ INITIALIZING PERCEPTRON MODEL: 
class Perceptron:                                       # Defining Perceptron. 
    def __init__(self, N, alpha=0.1):                   # Initializing Constructor Function. 
        self.W = np.random.randn(N + 1) / np.sqrt(N)    # Initializing Scaled Weight Matrix. 
        self.alpha = alpha                              # Initializing LR.
    
    def step(self, x):                                  # Defining Step Function. 
        return 1 if x > 0 else 0                        # Getting 1 if Positive else Negative. 
    
    def fit(self, X, y, epochs=10):                     # Defining Fit Function. 
        X = np.c_[X, np.ones((X.shape[0]))]             # Adding Column of Ones. 
        for epoch in np.arange(0, epochs):
            for (x, target) in zip(X, y):
                p = self.step(np.dot(x, self.W))        # Initializing Dot Product. 
                if p != target:
                    error = p - target                  # Computing Error. 
                    self.W += -self.alpha*error*x       # Updating Weight Matrix. 
    
    def predict(self, X, addBias=True):                 # Defining Predict Function. 
        X = np.atleast_2d(X)                            # Inspecting 2D Matrix. 
        if addBias:
            X = np.c_[X, np.ones((X.shape[0]))]         # Adding Bias Vector. 
        return self.step(np.dot(X, self.W))             # Initializing Dot Product. 

In [3]:
#@ EVALUATING PERCEPTRON BITWISE DATASETS: XOR:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])          # Initializing Array Example. 
y = np.array([[0], [1], [1], [1]])                      # Initializing Array Example. 

#@ TRAINING PERCEPTRON MODEL: 
p = Perceptron(X.shape[1], alpha=0.1)                   # Initializing Perceptron Model. 
p.fit(X, y, epochs=20)                                  # Training Model. 

#@ MODEL EVALUATION: 
for (x, target) in zip(X, y):
    pred = p.predict(x)
    print(f"data={x}, ground-truth={target}, pred={pred}")

data=[0 0], ground-truth=[0], pred=0
data=[0 1], ground-truth=[1], pred=1
data=[1 0], ground-truth=[1], pred=1
data=[1 1], ground-truth=[1], pred=1


**NEURAL NETWORKS:**

In [ ]:
#@ INITIALIZING BACKPROPAGATION ALGORITHM:
class NeuralNetwork:                                            # Defining Neural Network. 
    def __init__(self, layers, alpha=0.1):                      # Initializing Constructor Function. 
        self.W = []                                             # Initialization. 
        self.layers = layers                                    # Initialization. 
        self.alpha = alpha                                      # Initialization. 
        for i in np.arange(0, len(layers) - 1):
            w = np.random.randn(layers[i]+1, layers[i+1]+1)     # Initializing Weight Matrix. 
            self.W.append(w / np.sqrt(layers[i]))               # Normalizing Variance. 
        w = np.random.randn(layers[-2]+1, layers[-1])           # Initializing Weight Matrix. 
        self.W.append(w / np.sqrt(layers[-2]))                  # Normalizing Variance. 
    
    def __repr__(self):                                         # Function for Debugging. 
        return "NeuralNetwork: {}".format(
            "-".join(str(l) for l in self.layers))              # Inspecting. 
    
    def sigmoid(self, x):                                       # Defining Sigmoid Function. 
        return 1.0 / (1 + np.exp(-x))                           # Implementation of Sigmoid Function. 
    
    def sigmoid_deriv(self, x):                                 # Defining Function. 
        return x * (1 - x)                                      # Getting Derivative of Sigmoid. 
    
    def fit(X, y, epochs=1000, displayUpdate=100):              # Defining Fit Function. 
        X = np.c_[X, np.ones((X.shape[0]))]                     # Adding Column of Ones. 
        for epoch in np.arange(0, epochs):
            for (x, target) in zip(X, y):
                self.fit_parallel(x, target)                    # Backpropagation and Updating Matrix. 
            if epoch==0 or (epoch+1)%displayUpdate==0:
                loss = self.calculate_loss(X, y)                # Initializing Loss Calculation. 
                print("epoch={}, loss={:.7f}".format(
                    epoch+1, loss))                             # Inspecting Loss. 